In [1]:
#!/usr/bin/env python
# coding: utf-8

In [2]:
import time
import numpy
import datetime

In [3]:
import pandas as pd

In [4]:
topics = pd.read_csv('topics.csv')

In [5]:
d = {}

ar =topics.to_numpy()

for i in range(len(ar)):
    d[ar[i][0]] = int(sum(ar[i][1:101] ) == 0)
topics2 = pd.DataFrame(d.items(), columns=['topicId', 'text'])
topics2

,topicId,text
0,190991.0,0
1,2718.0,0
2,49923.0,0
3,124548.0,0
4,45289.0,0
...,...,...
206327,30555.0,0
206328,111715.0,0
206329,17614.0,0
206330,195949.0,1


In [6]:
idx_tp = set(sorted(topics['topicId'].tolist()))

In [7]:
train = pd.read_csv('train.csv')
top_feed1 = train[train['feedback'] == 'D'].reset_index()

top_feed1 = top_feed1.groupby('topicId')['feedback'].agg(['sum','count']).reset_index()

top_feed1.drop('sum', axis=1, inplace=True)
top_feed1.rename(columns={'count': 'D'}, inplace=True)


top_feed2 = train[train['feedback'] == 'L'].reset_index()

top_feed2 = top_feed2.groupby('topicId')['feedback'].agg(['sum','count']).reset_index()

top_feed2.drop('sum', axis=1, inplace=True)
top_feed2.rename(columns={'count': 'L'}, inplace=True)

top_feed = pd.merge(top_feed1, top_feed2, on="topicId")

top_feed

,topicId,D,L
0,0,54674,2111
1,1,42507,3442
2,2,35830,5044
3,3,39770,594
4,4,37174,2089
...,...,...,...
121107,166713,1,1
121108,166714,1,1
121109,166722,1,1
121110,166726,1,1


In [8]:
top_us1 = train[train['feedback'] == 'D'].reset_index()

top_us1 = top_us1.groupby('userId')['feedback'].agg(['sum','count']).reset_index()

top_us1.drop('sum', axis=1, inplace=True)
top_us1.rename(columns={'count': 'D'}, inplace=True)


top_us2 = train[train['feedback'] == 'L'].reset_index()

top_us2 = top_us2.groupby('userId')['feedback'].agg(['sum','count']).reset_index()

top_us2.drop('sum', axis=1, inplace=True)
top_us2.rename(columns={'count': 'L'}, inplace=True)

top_us = pd.merge(top_us1, top_us2, on="userId")

top_us['stavit_like'] = (top_us['L']/(top_us['L']+top_us['D']))**2
top_us['skolko_stavit'] = top_us['L']+top_us['D']
top_us.drop('D',axis=1,inplace=True)
top_us.drop('L',axis=1,inplace=True)
top_us

,userId,stavit_like,skolko_stavit
0,0,0.592129,859
1,1,0.620691,773
2,2,0.245209,727
3,3,0.385009,693
4,4,0.000423,632
...,...,...,...
2248064,2531608,0.250000,2
2248065,2531609,0.250000,2
2248066,2531611,0.250000,2
2248067,2531612,0.250000,2


In [9]:
users = pd.read_csv('users.csv')

In [10]:
infa = []
for el in (set(users['userId']) - set(top_us['userId'])):
    infa.append( [el,0.25,0])

In [11]:
about_users = pd.DataFrame(infa, columns=['userId','stavit_like','skolko_stavit'])

In [12]:
about_users

,userId,stavit_like,skolko_stavit
0,2097152,0.25,0
1,1048577,0.25,0
2,2097153,0.25,0
3,2097154,0.25,0
4,2097155,0.25,0
...,...,...,...
434183,2097139,0.25,0
434184,2097140,0.25,0
434185,1048565,0.25,0
434186,2097142,0.25,0


In [13]:
top_us

,userId,stavit_like,skolko_stavit
0,0,0.592129,859
1,1,0.620691,773
2,2,0.245209,727
3,3,0.385009,693
4,4,0.000423,632
...,...,...,...
2248064,2531608,0.250000,2
2248065,2531609,0.250000,2
2248066,2531611,0.250000,2
2248067,2531612,0.250000,2


In [14]:
rez_users = pd.concat([top_us,about_users],ignore_index=True)

In [15]:
rez_users

,userId,stavit_like,skolko_stavit
0,0,0.592129,859
1,1,0.620691,773
2,2,0.245209,727
3,3,0.385009,693
4,4,0.000423,632
...,...,...,...
2682252,2097139,0.250000,0
2682253,2097140,0.250000,0
2682254,1048565,0.250000,0
2682255,2097142,0.250000,0


In [16]:
del train
del topics

In [17]:
like_rate = dict()
popularity = dict()
for i,row in top_feed.iterrows():
    idx_tp.remove(row['topicId'])
    like_rate[row['topicId']] = row['L'] / (row['L'] + row['D'])
    popularity[row['topicId']] = (row['L'] + row['D'])

In [18]:
for el in idx_tp:
    like_rate[el] = 0.5
    popularity[el] = 0

In [19]:
like_r = pd.DataFrame(like_rate.items(), columns=['topicId', 'like_rate'])

In [20]:
popular = pd.DataFrame(popularity.items(), columns=['topicId', 'popularity'])

In [21]:
popular.info

<bound method DataFrame.info of         topicId  popularity
0             0       56785
1             1       45949
2             2       40874
3             3       40364
4             4       39263
...         ...         ...
206327   206327           0
206328   206328           0
206329   206329           0
206330   206330           0
206331   206331           0

[206332 rows x 2 columns]>

In [22]:
data = pd.read_csv('train.csv')

In [23]:
import numpy as np

In [24]:
data.reindex(np.random.permutation(data.index))

,topicId,userId,feedback
11883658,262,1180522,D
8417332,9493,229212,D
6401070,3894,318308,D
4136768,315,1508050,D
13273051,1196,2493885,D
...,...,...,...
5334517,3228,13594,D
4019580,1518,1835670,L
9115599,6023,167447,L
7860388,20955,2166895,L


In [25]:
#data = data.iloc[:9000000]

In [26]:
data.head()

,topicId,userId,feedback
0,198,1044368,L
1,154,1044368,L
2,6,1044368,D
3,77,1044368,D
4,81411,2604826,L


In [27]:
data.info

<bound method DataFrame.info of           topicId   userId feedback
0             198  1044368        L
1             154  1044368        L
2               6  1044368        D
3              77  1044368        D
4           81411  2604826        L
...           ...      ...      ...
14193799      231   703677        D
14193800      277  1095615        L
14193801       68  1095615        D
14193802    14621  1095615        L
14193803      203  1095615        D

[14193804 rows x 3 columns]>

In [28]:
data['feedback'] = data['feedback'].apply(lambda x: 1 if x == 'L' else 0)

In [29]:
data.head()

,topicId,userId,feedback
0,198,1044368,1
1,154,1044368,1
2,6,1044368,0
3,77,1044368,0
4,81411,2604826,1


In [30]:
data.info

<bound method DataFrame.info of           topicId   userId  feedback
0             198  1044368         1
1             154  1044368         1
2               6  1044368         0
3              77  1044368         0
4           81411  2604826         1
...           ...      ...       ...
14193799      231   703677         0
14193800      277  1095615         1
14193801       68  1095615         0
14193802    14621  1095615         1
14193803      203  1095615         0

[14193804 rows x 3 columns]>

In [31]:
users = pd.read_csv('users.csv')

In [32]:
users.head(50)
users['birthDate'] = users['birthDate'].fillna(8.581633e+08)

In [33]:
users['birthDate'] = users['birthDate'].apply(lambda x: int(float(abs(x))) if x >=0 else -int(float(abs(x))) )

In [34]:
def convert_time(date):
    return datetime.datetime(1970, 1, 1) + datetime.timedelta(milliseconds=date)

In [35]:
users['birthDate'] = users['birthDate'].apply(lambda x: convert_time(x))

In [36]:
def days_cnt(a):
    today = datetime.datetime(int(a[:4]),int(a[5:7]),int(a[8:10]))
    someday = datetime.datetime(1899, 1, 1)
    diff =   today-someday
    return diff.days

In [37]:
users['birthDate'] = users['birthDate'].apply(lambda x: days_cnt(str(x)))

In [38]:
import numpy as np

In [39]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    median = df_in[col_name].median()
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    def check_outlier(el):
        if el < fence_low or el >fence_high :
            return median
        return el
    df_in[col_name] = df_in[col_name].apply(lambda x:check_outlier(x) )
    return df_in
df = remove_outlier(users,'birthDate')
del users

In [40]:
min(df['birthDate'])

364

In [41]:
df['birthDate'] =df['birthDate'].apply(lambda x: x-10959)

In [42]:
max(df['birthDate'])

32372

In [43]:
#user_inf = dict()
#for i,row in df.iterrows():
#    user_inf[row['userId']] = [row['birthDate'],row['gender'],row['locationId']]

In [44]:
#check = 0
#for k,v in user_inf.items():
#    print(k,v)
#    if check == 10:
#        break
#    check += 1

In [45]:
df.head()

,userId,birthDate,gender,locationId
0,2019458,18653,2,5697551
1,96675,8658,2,12141127
2,1378979,20396,1,12141421
3,540187,19600,1,4126402
4,1182831,15053,2,2955426


In [46]:
result = data.merge(df, how='left', on="userId", sort=False)
result = result.merge(like_r, how='left', on="topicId", sort=False)
result = result.merge(popular, how='left', on="topicId", sort=False)
result = result.merge(rez_users, how='left', on="userId", sort=False)

result = result.merge(topics2, how='left', on="topicId", sort=False)



In [47]:
del data

In [48]:
result.head()

,topicId,userId,feedback,birthDate,gender,locationId,like_rate,popularity,stavit_like,skolko_stavit,text
0,198,1044368,1,9149,2,14882721,0.272987,7700,0.25,4,0
1,154,1044368,1,9149,2,14882721,0.403545,9253,0.25,4,0
2,6,1044368,0,9149,2,14882721,0.125470,37539,0.25,4,0
3,77,1044368,0,9149,2,14882721,0.205886,14304,0.25,4,0
4,81411,2604826,1,13380,1,2114999,0.500000,0,0.25,0,0


result.info

In [49]:
from catboost import Pool, CatBoostClassifier,CatBoostRegressor

In [50]:

result.drop('topicId',axis=1,inplace=True)
result.drop('locationId',axis=1,inplace=True)

In [51]:
result['gender'] =result['gender'].apply(lambda x: x-1)

In [52]:
result.head()

,userId,feedback,birthDate,gender,like_rate,popularity,stavit_like,skolko_stavit,text
0,1044368,1,9149,1,0.272987,7700,0.25,4,0
1,1044368,1,9149,1,0.403545,9253,0.25,4,0
2,1044368,0,9149,1,0.125470,37539,0.25,4,0
3,1044368,0,9149,1,0.205886,14304,0.25,4,0
4,2604826,1,13380,0,0.500000,0,0.25,0,0


In [53]:
train_labels = result['feedback'].values.tolist()

In [54]:
result.drop('feedback',axis=1,inplace=True)

In [55]:
result.head()

,userId,birthDate,gender,like_rate,popularity,stavit_like,skolko_stavit,text
0,1044368,9149,1,0.272987,7700,0.25,4,0
1,1044368,9149,1,0.403545,9253,0.25,4,0
2,1044368,9149,1,0.125470,37539,0.25,4,0
3,1044368,9149,1,0.205886,14304,0.25,4,0
4,2604826,13380,0,0.500000,0,0.25,0,0


In [56]:
cat_features = ['userId']
embedding_features = ['text']

In [57]:
result.drop('skolko_stavit',axis=1,inplace=True)
result.drop('stavit_like',axis=1,inplace=True)
result.drop('text',axis=1,inplace=True)


In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(result, train_labels, test_size=0.25, random_state=555)

In [60]:
del result

In [61]:
train_dataset = Pool(data=X_train,
                     label=y_train,
                    cat_features=cat_features)
#                   embedding_features=embedding_features)

In [62]:
eval_dataset = Pool(data=X_test[10000:],
                    label=y_test[10000:],
                    cat_features=cat_features)
#                   embedding_features=embedding_features)

In [105]:
model = CatBoostRegressor(iterations=200,
                           learning_rate=0.3,
                           task_type='GPU',
                           depth=10,
                          leaf_estimation_method='Gradient',
                           cat_features=cat_features)

In [64]:
test = pd.read_csv('test.csv')

In [65]:

test_read = test.merge(test.merge(df, how='left', on='userId', sort=False))
test_read = test_read.merge(rez_users, how='left', on="userId", sort=False)
test_read = test_read.merge(like_r, how='left', on='topicId', sort=False)
test_read = test_read.merge(popular, how='left', on='topicId', sort=False)
test_read = test_read.merge(topics2, how='left', on="topicId", sort=False)
test_read.drop('topicId',axis=1,inplace=True)
test_read.drop('locationId',axis=1,inplace=True)
test_read['gender'] =test_read['gender'].apply(lambda x: x-1)
test_read.drop('skolko_stavit',axis=1,inplace=True)
test_read.drop('stavit_like',axis=1,inplace=True)
del popular
del like_r
test_read.head()

,userId,birthDate,gender,like_rate,popularity,text
0,7056,11914,1,0.477444,266,0
1,7056,11914,1,0.974175,697,0
2,7056,11914,1,0.895833,48,0
3,7056,11914,1,0.797753,89,0
4,7056,11914,1,0.531073,177,0


In [66]:
test_read.head()

,userId,birthDate,gender,like_rate,popularity,text
0,7056,11914,1,0.477444,266,0
1,7056,11914,1,0.974175,697,0
2,7056,11914,1,0.895833,48,0
3,7056,11914,1,0.797753,89,0
4,7056,11914,1,0.531073,177,0


In [67]:
test_read.drop('text',axis=1,inplace=True)

In [68]:
X_train.head()

,userId,birthDate,gender,like_rate,popularity
11549474,4606,8625,0,0.928571,28
13049100,80882,20085,0,0.766667,60
7106183,400616,12204,1,0.784646,2006
7777538,186365,25777,1,0.725441,794
2329573,180074,9913,0,0.950995,653


In [69]:
del test

In [70]:
test_read.head()

,userId,birthDate,gender,like_rate,popularity
0,7056,11914,1,0.477444,266
1,7056,11914,1,0.974175,697
2,7056,11914,1,0.895833,48
3,7056,11914,1,0.797753,89
4,7056,11914,1,0.531073,177


In [106]:
model.fit(train_dataset)

0:	learn: 0.4562789	total: 416ms	remaining: 1m 22s
1:	learn: 0.4329070	total: 758ms	remaining: 1m 15s
2:	learn: 0.4208681	total: 1.1s	remaining: 1m 12s
3:	learn: 0.4093472	total: 1.43s	remaining: 1m 9s
4:	learn: 0.4033132	total: 1.72s	remaining: 1m 6s
5:	learn: 0.3978975	total: 2.01s	remaining: 1m 4s
6:	learn: 0.3939151	total: 2.37s	remaining: 1m 5s
7:	learn: 0.3910317	total: 2.64s	remaining: 1m 3s
8:	learn: 0.3884978	total: 2.94s	remaining: 1m 2s
9:	learn: 0.3865037	total: 3.29s	remaining: 1m 2s
10:	learn: 0.3844894	total: 3.68s	remaining: 1m 3s
11:	learn: 0.3829438	total: 3.98s	remaining: 1m 2s
12:	learn: 0.3801410	total: 4.4s	remaining: 1m 3s
13:	learn: 0.3785583	total: 4.64s	remaining: 1m 1s
14:	learn: 0.3771440	total: 5.05s	remaining: 1m 2s
15:	learn: 0.3759807	total: 5.31s	remaining: 1m 1s
16:	learn: 0.3746962	total: 5.68s	remaining: 1m 1s
17:	learn: 0.3736847	total: 5.93s	remaining: 60s
18:	learn: 0.3732871	total: 6.28s	remaining: 59.8s
19:	learn: 0.3722845	total: 6.58s	remainin

161:	learn: 0.3556647	total: 53s	remaining: 12.4s
162:	learn: 0.3556514	total: 53.3s	remaining: 12.1s
163:	learn: 0.3556183	total: 53.6s	remaining: 11.8s
164:	learn: 0.3555963	total: 53.9s	remaining: 11.4s
165:	learn: 0.3555599	total: 54.2s	remaining: 11.1s
166:	learn: 0.3555396	total: 54.4s	remaining: 10.8s
167:	learn: 0.3555023	total: 54.8s	remaining: 10.4s
168:	learn: 0.3554878	total: 55.1s	remaining: 10.1s
169:	learn: 0.3554447	total: 55.4s	remaining: 9.77s
170:	learn: 0.3554078	total: 55.7s	remaining: 9.44s
171:	learn: 0.3553860	total: 55.9s	remaining: 9.11s
172:	learn: 0.3553534	total: 56.3s	remaining: 8.78s
173:	learn: 0.3553450	total: 56.6s	remaining: 8.46s
174:	learn: 0.3552953	total: 56.9s	remaining: 8.13s
175:	learn: 0.3552789	total: 57.2s	remaining: 7.8s
176:	learn: 0.3552640	total: 57.5s	remaining: 7.47s
177:	learn: 0.3552481	total: 57.8s	remaining: 7.14s
178:	learn: 0.3552434	total: 58.2s	remaining: 6.82s
179:	learn: 0.3552326	total: 58.5s	remaining: 6.5s
180:	learn: 0.35

In [107]:
testing_dataset = Pool(data=X_test[:10000],
                    label=y_test[:10000],
                    cat_features=cat_features)
#                   embedding_features=embedding_features
preds_class = model.predict(testing_dataset)

In [108]:
preds_class[:50]

array([ 0.44075358,  0.98057892,  0.39739591,  0.78741358,  1.03492621,
        0.99928556,  0.73981845,  0.80627444,  0.10399244,  0.80248843,
        0.3026183 , -0.02103755,  0.04313963,  0.67493056,  0.9680758 ,
        0.75643954,  0.89367225,  0.75686172,  0.05149152,  0.94945164,
        0.9901769 ,  0.68696422, -0.10777485,  0.7606626 , -0.01445819,
        0.02181224,  0.53927523,  0.18319215,  0.89407353,  0.08031692,
        0.87185521,  0.40471099,  0.07907604,  0.70199753,  0.64443   ,
        0.97014937,  0.14728026,  0.97323002,  0.15523272,  0.12094784,
        0.05696921,  0.04302703,  1.05637189,  0.02355968,  0.13552956,
        0.17729378,  0.125334  ,  0.4295531 ,  0.61737106,  0.04448732])

In [109]:
preds_class = [int(el >= 0.5) for el in preds_class]

In [110]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [111]:
accuracy_score(y_test[:10000], preds_class)

0.8354

In [112]:
test_read.info

<bound method DataFrame.info of           userId  birthDate  gender  like_rate  popularity
0           7056      11914       1   0.477444         266
1           7056      11914       1   0.974175         697
2           7056      11914       1   0.895833          48
3           7056      11914       1   0.797753          89
4           7056      11914       1   0.531073         177
...          ...        ...     ...        ...         ...
1578797   468755      13022       1   0.846863        3905
1578798   692556      16832       1   0.953125          64
1578799  1078639       7389       1   0.186501        3630
1578800   184614       8704       0   0.082103        1084
1578801   184614       8704       0   0.736842          76

[1578802 rows x 5 columns]>

In [113]:
test_read.head()

,userId,birthDate,gender,like_rate,popularity
0,7056,11914,1,0.477444,266
1,7056,11914,1,0.974175,697
2,7056,11914,1,0.895833,48
3,7056,11914,1,0.797753,89
4,7056,11914,1,0.531073,177


In [114]:
test_pred = []
l= 0
for r in range(10000,1578802,10000):
    check_test = Pool(data=test_read.iloc[l:r],
                     cat_features=cat_features)
    l = r
    for el in model.predict(check_test):
        test_pred.append(int(el >= 0.5))
    print(len(test_pred))
check_test = Pool(data=test_read.iloc[1570000:],
                     cat_features=cat_features)
for el in model.predict(check_test):
    test_pred.append(int(el >= 0.5))
print(len(test_pred))    

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [80]:
#super_test = Pool(data=test_read,
#                    cat_features=cat_features)
#                   embedding_features=embedding_features)

In [81]:
for i in range()

SyntaxError: invalid syntax (1816844243.py, line 1)

In [ ]:
#test_pred = model.predict(super_test)

In [115]:
print(test_pred[:100])

[0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]


In [121]:
def return_name(el):
    if el  >= 0.5:
        return 'L'
    return 'D'

In [122]:
rez_test = [return_name(el) for el in test_pred ]

In [123]:
rez_test[:100]

['D',
 'L',
 'L',
 'L',
 'D',
 'D',
 'D',
 'L',
 'L',
 'D',
 'D',
 'D',
 'L',
 'D',
 'D',
 'D',
 'L',
 'L',
 'L',
 'D',
 'L',
 'L',
 'L',
 'L',
 'D',
 'L',
 'L',
 'D',
 'L',
 'D',
 'L',
 'L',
 'D',
 'D',
 'D',
 'D',
 'D',
 'L',
 'L',
 'L',
 'D',
 'L',
 'D',
 'D',
 'L',
 'L',
 'D',
 'D',
 'D',
 'L',
 'L',
 'L',
 'D',
 'L',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'L',
 'L',
 'D',
 'L',
 'D',
 'L',
 'D',
 'D',
 'L',
 'D',
 'L',
 'D',
 'L',
 'D',
 'L',
 'D',
 'D',
 'L',
 'L',
 'L',
 'D',
 'L',
 'L',
 'L',
 'L',
 'D',
 'L',
 'D',
 'D',
 'L',
 'D',
 'L',
 'L',
 'L',
 'L',
 'D',
 'D',
 'L']

In [124]:
with open('send.txt','w') as f:
    for el in rez_test:
        f.write(el+'\n')

In [125]:
print(len(rez_test))

1578802
